# 1 этап проекта: сбор данных.
Работу выполнила: Политова Ирина, группа БКЛ-203

**Данные:** Я буду работать с сайтом https://wheatoncollege.edu/vive-voix/titres/, буду собирать с него название, автора и текст стихотворений.  
  
**Таблица на выходе**: Таблица будет содержать столбцы id, title, author, link (ссылка на стихотворение), poem (текст стихотворения), audio (ссылка на аудио).

In [41]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import csv

In [42]:
session = requests.session()

Начнем обрабатывать страницы сайта. Сделаем первую программу, которая будет парсить главную страницу со списком всех стихотворений и доставать оттуда названия, авторов и ссылки на стихотворения. Итог - список, где элементы имеют вид "[название, автор, ссылка]".

In [43]:
url_page = 'https://wheatoncollege.edu/vive-voix/titres/'

all_poems = []

req = session.get(url_page, headers={'User-Agent': UserAgent().chrome})
page = req.text
soup = BeautifulSoup(page, 'html.parser')

main = soup.find('main')

all_li = main.find_all('li')

for li in all_li:
    a = li.find_all('a')
    title = a[0].text
    link = 'https://wheatoncollege.edu/vive-voix/titres/' + a[0].attrs['href']
    author = a[1].text
    
    if author.startswith(' ('):
        author = author[2:]
    
    new_poem = []
    new_poem.append(title)
    new_poem.append(author)
    new_poem.append(link)
    all_poems.append(new_poem)

print(all_poems[100])

['Dimanche après-midi', 'Paul ÉLUARD', 'https://wheatoncollege.edu/vive-voix/titres//dimanche-aprsmidi/']


Отлично, всё работает! Наш следующий шаг - научиться парсить отдельное стихотворение, имея на входе ссылку на него. Напишем функцию vers, которая берет на вход ссылку и достает по ней текст стихотворения и аудио в формате списка [текст, аудио].

In [44]:
def vers(url_vers):
    
    vers_data = []
    all_poem = []

    req = session.get(url_vers, headers={'User-Agent': UserAgent().chrome})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    
    main = soup.find('main')

    poem = main.find_all('p')
    for p in poem:
        all_poem.append(p.text)

    a = main.find('a')
    audio = a.attrs['href']
    
    vers_data.append(all_poem)
    vers_data.append(audio)
    
    return vers_data

In [45]:
pprint(vers('https://wheatoncollege.edu/vive-voix/titres/streets/'))

[['\xa0\xa0\xa0\xa0Dansons la gigue!',
  'J’aimais surtout ses jolis yeux\n'
  'Plus clairs que l’étoile des cieux,\n'
  'J’aimais ses yeux malicieux.',
  '\xa0\xa0\xa0\xa0Dansons la gigue!',
  'Elle avait des façons vraiment\n'
  'De désoler un pauvre amant,\n'
  'Que c’en était vraiment charmant!',
  '\xa0\xa0\xa0\xa0Dansons la gigue!',
  'Mais je trouve encore meilleur\n'
  'Le baiser de sa bouche en fleur\n'
  'Depuis qu’elle est morte à mon coeur.',
  '\xa0\xa0\xa0\xa0Dansons la gigue!',
  'Je me souviens, je me souviens\n'
  'Des heures et des entretiens,\n'
  'Et c’est le meilleur de mes biens.',
  '\xa0\xa0\xa0\xa0Dansons la gigue!',
  'Soho.\n'
  'II\n'
  'O la rivière dans la rue!\n'
  'Fantastiquement apparue\n'
  'Derrière un mur haut de cinq pieds,\n'
  'Elle roule sans un murmure\n'
  'Son onde opaque et pourtant pure\n'
  'Par les faubourgs pacifiés.',
  'La chaussée est très large, en sorte\n'
  'Que l’eau jaune comme une morte\n'
  'Dévale ample et sans nuls espoirs\n'

In [46]:
errors = [] # сюда будут сложены ошибки

for poem in all_poems:
    
    url_vers = poem[2]
    
    try:
        new_data = vers(url_vers)
        poem.append(new_data[0])
        poem.append(new_data[1])

    except:
        errors.append(url_vers)

print(errors)

['https://wheatoncollege.edu/vive-voix/titres//beyrouth/', 'https://wheatoncollege.edu/vive-voix/titres//dites-nous-femmes/', 'https://wheatoncollege.edu/vive-voix/titres//lheure-viendra/', 'https://wheatoncollege.edu/vive-voix/titres//il-est-des-enfants/', 'https://wheatoncollege.edu/vive-voix/titres//je-vous-salue/', 'https://wheatoncollege.edu/vive-voix/titres//le-nomade/', 'https://wheatoncollege.edu/vive-voix/titres//non-homme/', 'https://wheatoncollege.edu/vive-voix/titres//les-reves-viennent-mourir-sur-la-plage/']


Нашлись 8 стихотворений без аудио. Не страшно, просто удалим их из нашего списка стихотворений.

In [47]:
for e in errors:
    
    for poem in all_poems:
        
        if poem[2] == e:
            all_poems.remove(poem)

In [48]:
all_poems[101]

['Don du poème',
 'Stéphane MALLARMÉ',
 'https://wheatoncollege.edu/vive-voix/titres//don-du-pome/',
 ['Je t’apporte l’enfant d’une nuit d’Idumée !\nNoire, à l’aile saignante et pâle, déplumée,\nPar le verre brûlé d’aromates et d’or,\nPar les carreaux glacés, hélas ! mornes encor,\nL’aurore se jeta sur la lampe angélique.\nPalmes ! et quand elle a montré cette relique\nA ce père essayant un sourire ennemi,\nLa solitude bleue et stérile a frémi.\nO la berceuse avec ta fille et l’innocence\nDe vos pieds froids, accueille une horrible naissance\nEt, ta voix rappelant viole et clavecin,\nAvec le doigt fané presseras-tu le sein\nPar qui coule en blancheur sibylline la femme\nPour des lèvres que l’air du vierge azur affame ?',
  'dit par Claude MONTEUX',
  'téphane MALLARMÉ (Vers et prose, 1893)'],
 'https://wheatoncollege.edu/wp-content/uploads/2011/01/dondupoeme.mp3']

Итак, что у нас имеется: список из 296 элементов, каждый из которых представляет собой список [название, автор, ссылка, текст, аудио] одного стихотворения. Дальше будем заполнять таблицу, перед этим переведем наш список в список словарей.

In [49]:
poems = []

for poem in all_poems:
    d = {}
    d['title'] = poem[0]
    d['author'] = poem[1]
    d['link'] = poem[2]
    
    poem_start = poem[3]
    all_poem = []
    text_poem = ''
    for p in poem_start:
        s_poem = p.strip('\n')
        all_poem.append(s_poem)
    
    for p in all_poem:
        text_poem += str(p) + '\n'
    
    d['poem'] = text_poem
    d['audio'] = poem[4]
    poems.append(d)

In [50]:
pprint(poems[5])

{'audio': 'https://wheatoncollege.edu/wp-content/uploads/2011/01/albatros.mp3',
 'author': 'Charles BAUDELAIRE',
 'link': 'https://wheatoncollege.edu/vive-voix/titres//lalbatros/',
 'poem': 'Souvent, pour s’amuser, les hommes d’équipage\n'
         'Prennent des albatros, vastes oiseaux des mers,\n'
         'Qui suivent, indolents compagnons de voyage,\n'
         'Le navire glissant sur les gouffres amers.\n'
         'A peine les ont-ils déposés sur les planches,\n'
         'Que ces rois de l’azur, maladroits et honteux,\n'
         'Laissent piteusement leurs grandes ailes blanches\n'
         'Comme des avirons traîner à côté d’eux.\n'
         'Ce voyageur ailé, comme il est gauche et veule !\n'
         'Lui, naguère si beau, qu’il est comique et laid !\n'
         'L’un agace son bec avec un brûle-gueule,\n'
         'L’autre mime, en boitant, l’infirme qui volait !\n'
         'Le Poète est semblable au prince des nuées\n'
         'Qui hante la tempête et se rit de l’archer 

In [51]:
with open('french_poems.csv', 'w', encoding='utf-8', newline='') as csvfile:
    fieldnames = ['title', 'author', 'link', 'poem', 'audio']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    
    for poem in poems:
        writer.writerow(poem)

Ура, у нас всё готово!!! Теперь у нас есть таблица, в которую всё вписано.